In [2]:
! pip install einops accelerate bitsandbytes

In [6]:
import torch
torch.cuda.empty_cache()

In [1]:
from transformers import AutoModelForCausalLM, AutoProcessor, BitsAndBytesConfig
import torch

CUDA = 'cuda:1'  # 사용하고자 하는 GPU

# load the processor
processor = AutoProcessor.from_pretrained(
    'allenai/Molmo-7B-D-0924',
    trust_remote_code=True,  # 원격에서 커스텀 코드를 가져와 실행
    torch_dtype='auto',
    device_map={"": CUDA}  # 명시적으로 device_map을 설정
)

# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4비트로 로드
    bnb_4bit_use_double_quant=True,  # 더블 양자화 사용
    bnb_4bit_quant_type="nf4",  # 양자화 유형 선택 (nf4 또는 fp4)
    bnb_4bit_compute_dtype=torch.float16  # 계산을 위한 dtype
)

# load the model
model = AutoModelForCausalLM.from_pretrained(
    'allenai/Molmo-7B-D-0924',
    trust_remote_code=True,  # 원격에서 커스텀 코드를 가져와 실행
    torch_dtype=torch.float16,  # 명시적으로 float16 타입으로 설정
    quantization_config=bnb_config,  # 양자화 설정 전달
    device_map={"": CUDA},  # model을 cuda:1에서 실행하도록 설정
)

/home/metaai1/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 7/7 [00:03<00:00,  1.81it/s]


In [2]:

# process the image and text
inputs = processor.process(
    images=[Image.open(requests.get("https://picsum.photos/id/237/536/354", stream=True).raw)],
    # text="Describe this image."
    text="Please tell me the overall color and additional colors of the animal in this photo and write a sentence that extracts the characteristics of the fur, such as color combinations and characteristics."
)

In [3]:

# move inputs to the correct device and make a batch of size 1
inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}


In [4]:
## 101분 20초 걸림
## with torch.autocast(device_type="cuda", enabled=True, dtype=torch.bfloat16):
##   output = model.generate_from_batch(
##       inputs,
##       GenerationConfig(max_new_tokens=200, stop_strings="<|endoftext|>"),
##       tokenizer=processor.tokenizer
##   )

In [4]:
print('추론 중...')
output = model.generate_from_batch(
    inputs,
    GenerationConfig(max_new_tokens=200, stop_strings="<|endoftext|>"),
    tokenizer=processor.tokenizer
)
print('추론 완료...')

추론 중...
추론 완료...


In [5]:

# only get generated tokens; decode them to text
generated_tokens = output[0,inputs['input_ids'].size(1):]
generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)

# print the generated text
print(generated_text)

# >>>  This image features an adorable black Labrador puppy, captured from a top-down
#      perspective. The puppy is sitting on a wooden deck, which is composed ...


 The overall color of the animal in this photo is black. There are no additional colors visible on its fur.

The puppy's fur is a solid, glossy black color, covering its entire body. The fur appears to be short and sleek, giving the puppy a sleek and elegant appearance. The black coat stands out beautifully against the light brown wooden floor, creating a striking contrast that emphasizes the puppy's glossy coat.
